# Interpolation

PODPAC supports the following interpolation methods with the associated Interpolator classes:
    
```python
INTERPOLATION_METHODS = {
    'nearest': [NearestNeighbor],
    'nearest_preview': [NearestPreview],
    'bilinear':[Rasterio, Scipy],
    'cubic':[Rasterio, Scipy],
    'cubic_spline':[Rasterio, Scipy],
    'lanczos':[Rasterio, Scipy],
    'average':[Rasterio, Scipy],
    'mode':[Rasterio, Scipy],
    'gauss':[Rasterio, Scipy],
    'max':[Rasterio, Scipy],
    'min':[Rasterio, Scipy],
    'med':[Rasterio, Scipy],
    'q1':[Rasterio, Scipy],
    'q3': [Rasterio, Scipy]
}
```

In [18]:
# notebook imports
from IPython.display import display
import numpy as np
from podpac.core.coordinates import Coordinates, clinspace
from podpac.core.data.types import Array

## Demo Setup

Create a simple Array data source node and alternate coordinates to evaluate

In [19]:
# data source 
source = np.random.rand(3, 3)
native_coords = Coordinates([[0, 5, 10], clinspace('2018-01-01', '2018-01-18', 3)], 
                             dims=['lat', 'time'])
node = Array(source=source, native_coordinates=native_coords)

node.source

array([[0.89863362, 0.9154702 , 0.72588443],
       [0.53842186, 0.22045984, 0.64350443],
       [0.99793058, 0.5996738 , 0.99236829]])

In [20]:
# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 9], clinspace('2018-01-01', '2018-01-09', 3)], 
                          dims=['lat', 'time'])

## Nearest Neighbor

In [21]:
# simple interpolation definition
node.interpolation = 'nearest'

In [22]:
# evaluate at coords with nearest neighbor interpolation
output = node.eval(eval_coords)

# elements are chosen based on their nearest neighbor
output.data

array([[0.89863362, 0.9154702 , 0.9154702 ],
       [0.89863362, 0.9154702 , 0.9154702 ],
       [0.99793058, 0.5996738 , 0.5996738 ]])

#### Spatial Tolerance

In [14]:
# set a space tolerance on interpolation
node.interpolation = {
    'method': 'nearest',
    'params': {
        'spatial_tolerance': 1.1
    }
}

output = node.eval(eval_coords)

# the 1st element is outside of the space tolerance
output.data

array([[0.8151018 , 0.84889048, 0.84889048],
       [       nan,        nan,        nan],
       [0.26857165, 0.18086109, 0.18086109]])

#### Time Tolerance

In [19]:
# set a time tolerance on interpolation
node.interpolation = {
        'method': 'nearest',
        'params': {
            'spatial_tolerance': 1.1,
            'time_tolerance': np.timedelta64(1, 'D')
        }
}

output = node.eval(eval_coords)
output.data

array([[0.8151018 ,        nan, 0.84889048],
       [       nan,        nan,        nan],
       [0.26857165,        nan, 0.18086109]])

## Rasterio Interpolation

Rasterio provides support for the following interpolation methods:


```python
['nearest', 
 'bilinear', 
 'cubic', 
 'cubic_spline', 
 'lanczos', 
 'average', 
 'mode', 
 'gauss', 
 'max', 
 'min', 
 'med', 
 'q1', 
 'q3']
```

In [23]:
# data source 
source = np.random.rand(5, 5)
native_coords = Coordinates([np.linspace(0, 10, 5), np.linspace(0, 10, 5)], 
                             dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

node.source

array([[0.60156958, 0.54198521, 0.90493734, 0.68843579, 0.8564641 ],
       [0.49695162, 0.27169023, 0.27647436, 0.94737437, 0.0887236 ],
       [0.54245185, 0.22610308, 0.48760185, 0.53948611, 0.27071314],
       [0.40008639, 0.06394548, 0.51670318, 0.72545895, 0.1061752 ],
       [0.65065352, 0.64087531, 0.81125917, 0.65095995, 0.00435908]])

In [24]:
# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 5, 9, 10], [.7, 1.2, 5, 9, 10]], 
                          dims=['lat', 'lon'])

In [26]:
# use an interpolation method from rasterio
node.interpolation = 'max'
output = node.eval(eval_coords)

# elements are chosen based on their nearest neighbor
output.data

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])